<a href="https://colab.research.google.com/github/TheSpaceMan915/Artificial-Intelligence/blob/main/Lab8_Recommender_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Лабораторная работа №8  Колычев Никита  ВПР34**

In [ ]:
#@title Imports
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

# **Preparing the dataset**

In [ ]:
# reading the datasets
path_folder = "/content/drive/MyDrive/Colab Notebooks/AI/Datasets/Amazon Books/"
df_books = pd.read_csv(path_folder + "books.csv", on_bad_lines="skip", 
                       encoding="latin-1", nrows=25000, sep=";")
df_book_ratings = pd.read_csv(path_folder + "book_ratings.csv", on_bad_lines="skip", 
                              encoding="latin-1", nrows=25000, sep=";")

In [ ]:
df_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
df_book_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
# creating another dataframe for all data
# df_merged = pd.merge(df_book_ratings, df_copy, how="inner", left_on="ISBN", right_on="ISBN")
# df_merged.head()

# creating a dataframe with book titles
df_copy = df_books.filter(["ISBN", "Book-Title"])
df_copy.head()

,ISBN,Book-Title
0,0195153448,Classical Mythology
1,0002005018,Clara Callan
2,0060973129,Decision in Normandy
3,0374157065,Flu: The Story of the Great Influenza Pandemic...
4,0393045218,The Mummies of Urumchi


# **Creating a feedback matrix**

In [ ]:
#  reshaping df_merged
matrix_feedback = df_book_ratings.pivot(index="ISBN", columns="User-ID", values="Book-Rating")
# matrix_feedback.head()

In [ ]:
matrix_feedback.fillna(0, inplace=True)
print("shape:", matrix_feedback.shape)
matrix_feedback.head()

shape: (20833, 3003)


User-ID,2,7,8,9,10,12,14,16,17,19,...,278832,278836,278838,278843,278844,278846,278849,278851,278852,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
00 14 02 5314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002005018,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000221122X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002231115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002232766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# **Cleaning the matrix**

In [ ]:
# counting the amount of votes for each book
series_amount_votes_by_book = df_book_ratings.groupby("ISBN")["User-ID"].agg("count")
series_amount_votes_by_book.head()

ISBN
00 14 02 5314    1
0002005018       1
000221122X       1
0002231115       1
0002232766       1
Name: User-ID, dtype: int64

In [ ]:
# counting the amount of votes for each user
series_amount_votes_by_user = df_book_ratings.groupby("User-ID")["ISBN"].agg("count")
series_amount_votes_by_user.head()

User-ID
2      1
7      1
8     18
9      3
10     2
Name: ISBN, dtype: int64

# **Creating a compressed sparse row matrix**

In [ ]:
# finding appropriate books and users
mask_index_users = series_amount_votes_by_user[series_amount_votes_by_user > 1].index
mask_index_books = series_amount_votes_by_book[series_amount_votes_by_book > 1].index

# applying the masks
matrix_filtered = matrix_feedback.loc[:, mask_index_users]
matrix_filtered = matrix_filtered.loc[mask_index_books, :]
print("shape:", matrix_filtered.shape)
matrix_filtered.head()

shape: (2313, 1255)


User-ID,8,9,10,14,16,17,22,26,32,38,...,278799,278818,278832,278838,278843,278844,278846,278849,278851,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
0006170064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0006174558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0006513204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0006513409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0006550789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
matrix_compressed = csr_matrix(matrix_filtered)
print(matrix_compressed)

  (0, 396)	10.0
  (0, 827)	7.0
  (1, 824)	8.0
  (2, 196)	7.0
  (3, 599)	4.0
  (4, 906)	5.0
  (6, 139)	10.0
  (7, 79)	9.0
  (7, 525)	5.0
  (11, 449)	5.0
  (12, 1161)	10.0
  (13, 492)	8.0
  (13, 989)	10.0
  (13, 1052)	10.0
  (13, 1082)	5.0
  (14, 1082)	9.0
  (15, 148)	8.0
  (16, 331)	8.0
  (17, 254)	5.0
  (17, 1153)	7.0
  (18, 645)	10.0
  (19, 587)	7.0
  (19, 648)	7.0
  (21, 539)	8.0
  (21, 684)	9.0
  :	:
  (2275, 797)	9.0
  (2277, 715)	7.0
  (2278, 508)	5.0
  (2281, 797)	9.0
  (2282, 981)	6.0
  (2284, 981)	8.0
  (2285, 432)	7.0
  (2285, 797)	8.0
  (2287, 590)	10.0
  (2290, 715)	6.0
  (2291, 69)	3.0
  (2291, 715)	7.0
  (2293, 1142)	10.0
  (2294, 382)	7.0
  (2295, 1218)	10.0
  (2296, 981)	8.0
  (2297, 981)	6.0
  (2303, 607)	8.0
  (2303, 715)	7.0
  (2304, 699)	7.0
  (2305, 504)	8.0
  (2306, 699)	6.0
  (2307, 989)	10.0
  (2309, 723)	9.0
  (2310, 723)	10.0


In [ ]:
# transforming "ISBN" indexes to a column
matrix_filtered = matrix_filtered.rename_axis(None, axis=1).reset_index()
matrix_filtered.head()

,ISBN,8,9,10,14,16,17,22,26,32,...,278799,278818,278832,278838,278843,278844,278846,278849,278851,278854
0,0006170064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0006174558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0006513204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0006513409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0006550789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# **Finding books and their id**

In [ ]:
# training the model
nearest_neighbors = NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=20, n_jobs=-1)
nearest_neighbors.fit(matrix_compressed)

# looking for Harry Potter books
mask = df_copy["Book-Title"].str.contains("Harry Potter and")
df_found = df_copy[mask]
df_found.head()

,ISBN,Book-Title
2143,059035342X,Harry Potter and the Sorcerer's Stone (Harry P...
2809,0590353403,Harry Potter and the Sorcerer's Stone (Book 1)
3459,0439064872,Harry Potter and the Chamber of Secrets (Book 2)
3839,0439136350,Harry Potter and the Prisoner of Azkaban (Book 3)
5431,0439139597,Harry Potter and the Goblet of Fire (Book 4)


In [ ]:
# finding the id from the dataframe of books
id_book = df_found.iloc[1]["ISBN"]
print("id_book:", id_book)

id_book: 0590353403


In [ ]:
# finding the index of the book in the matrix of recommendations
mask = matrix_filtered["ISBN"] == id_book
index_book_in_matrix = matrix_filtered[mask].index[0]
print("id_book_in_matrix:", index_book_in_matrix)

id_book_in_matrix: 1510


In [ ]:
# finding the ratings of different users for the book
matrix_ratings = matrix_compressed[index_book_in_matrix]
print(matrix_ratings)

  (0, 42)	9.0
  (0, 315)	9.0
  (0, 341)	10.0
  (0, 757)	8.0


# **Getting recommendations**

In [ ]:
# finding distances and their indexes
amount_recommendations = 10
arr_distances, arr_indexes = nearest_neighbors.kneighbors(matrix_ratings, n_neighbors=amount_recommendations+1)

# placing data into one list(by removing nested lists)
list_indexes = arr_indexes.squeeze().tolist()
list_distances = arr_distances.squeeze().tolist()
list_tuples = list(zip(list_indexes, list_distances))
print("List tuples:", list_tuples)

List tuples: [(1510, 0.0), (933, 0.16738272072798577), (927, 0.16879821743580914), (930, 0.2415697759583324), (1574, 0.25590257251144055), (2032, 0.4461512243782887), (281, 0.4461512243782887), (1673, 0.4461512243782887), (1249, 0.5015361019404598), (1208, 0.5015361019404598), (1453, 0.5015361019404598)]


In [ ]:
# sorting the list of tuples by distances
list_tuples_sorted = sorted(list_tuples, key=lambda x: x[1], reverse=False)

# deleting the first tuple because it is the book I've chosen
list_tuples_sorted.pop(0)
print("List tuples:", list_tuples_sorted)

List tuples: [(933, 0.16738272072798577), (927, 0.16879821743580914), (930, 0.2415697759583324), (1574, 0.25590257251144055), (2032, 0.4461512243782887), (281, 0.4461512243782887), (1673, 0.4461512243782887), (1249, 0.5015361019404598), (1208, 0.5015361019404598), (1453, 0.5015361019404598)]


In [ ]:
list_recommendations = []
for element in list_tuples_sorted:
  # finding ISBN of a recommended book in matrix_filtered
  id_book_in_matrix = matrix_filtered.iloc[element[0]]["ISBN"]

  # finding the index of a book in df with book titles
  index_book = df_copy[df_copy["ISBN"] == id_book_in_matrix].index

  # getting the title of a book
  title_book = df_copy.iloc[index_book]["Book-Title"].values[0]
  distance_book = element[1]
  
  # putting the title and distance in the list as a dictionary
  list_recommendations.append({"Title": title_book, "Distance": distance_book})

# creating a dataframe
df_recommendations = pd.DataFrame(list_recommendations, index=range(amount_recommendations))
df_recommendations.head()

,Title,Distance
0,Harry Potter and the Goblet of Fire (Book 4),0.167383
1,Harry Potter and the Chamber of Secrets (Book 2),0.168798
2,Harry Potter and the Prisoner of Azkaban (Book 3),0.241570
3,The Hobbit: or There and Back Again,0.255903
4,Dogbert's Top Secret Management Handbook,0.446151
